#  Exercise 01
The goal is to find the best set of hyper-parameters which maximize the
performance on a training set.

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# This line is currently required to import HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

from scipy.stats import expon, uniform
from scipy.stats import randint

df = pd.read_csv(
    "https://www.openml.org/data/get_csv/1595261/adult-census.csv")
# Or use the local copy:
# df = pd.read_csv('../datasets/adult-census.csv')

target_name = "class"
target = df[target_name].to_numpy()
data = df.drop(columns=target_name)

df_train, df_test, target_train, target_test = train_test_split(
    data, target, random_state=42)

You should:
- create a preprocessor using an `OrdinalEncoder`
- use a `HistGradientBoostingClassifier` to make predictions
- use a `RandomizedSearchCV` to find the best set of hyper-parameters by
  tuning the following parameters: `learning_rate`, `l2_regularization`,
  `max_leaf_nodes`, and `min_samples_leaf`.

In [2]:
ordinal_encoding_columns = [
    'workclass', 'education', 'marital-status', 'occupation',
    'relationship', 'race', 'native-country', 'sex']

categories = [
    data[column].unique()
    for column in data[ordinal_encoding_columns]]

preprocessor = ColumnTransformer([
    ('ordinal-encoder', OrdinalEncoder(categories=categories),
     ordinal_encoding_columns)], remainder='passthrough',
                                 sparse_threshold=0)

model = Pipeline([
    ('preprocessor', preprocessor),
    ('gbrt', HistGradientBoostingClassifier(max_iter=50))])
param_distributions = {
    'gbrt__learning_rate': expon(loc=0.001, scale=0.5),
    'gbrt__l2_regularization': uniform(loc=0, scale=0.5),
    'gbrt__max_leaf_nodes': randint(5, 30),
    'gbrt__min_samples_leaf': randint(5, 30)}
model_grid_search = RandomizedSearchCV(
    model, param_distributions=param_distributions, n_iter=10,
    n_jobs=4)
model_grid_search.fit(df_train, target_train)
print(
    f"The accuracy score using a {model_grid_search.__class__.__name__} is "
    f"{model_grid_search.score(df_test, target_test):.2f}")
print(
    f"The best set of parameters is: {model_grid_search.best_params_}"
)

/home/lesteve/miniconda3/envs/scikit-learn-tutorial/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


The accuracy score using a RandomizedSearchCV is 0.88
The best set of parameters is: {'gbrt__l2_regularization': 0.0996800489707867, 'gbrt__learning_rate': 0.1658500102217152, 'gbrt__max_leaf_nodes': 21, 'gbrt__min_samples_leaf': 19}


In [3]:
df_results = pd.DataFrame(model_grid_search.cv_results_)
columns = (['mean_test_score', 'std_test_score'] +
           [col for col in df_results.columns if 'param_' in col])
df_results.sort_values(by='mean_test_score',
                       ascending=False)[columns]

,mean_test_score,std_test_score,param_gbrt__l2_regularization,param_gbrt__learning_rate,param_gbrt__max_leaf_nodes,param_gbrt__min_samples_leaf
6,0.869537,0.000486,0.09968,0.16585,21,19
8,0.868909,0.001318,0.165328,0.165889,19,28
7,0.868636,0.000966,0.217512,0.233881,13,23
9,0.867680,0.000440,0.146045,0.219542,8,21
2,0.866425,0.001347,0.0161534,0.458746,23,29
5,0.865797,0.001562,0.250776,0.574314,15,28
3,0.865715,0.001386,0.210573,0.472672,28,29
0,0.860364,0.000765,0.118935,0.62203,27,11
1,0.855696,0.000421,0.329013,1.0929,14,13
4,0.840135,0.002817,0.218251,0.0231929,11,25
